# HW 3 Problem 1: Investigating Mixed Selectivity


In this problem, we will be working with data from the [Hardcastle](https://www.cell.com/action/showPdf?pii=S0896-6273%2817%2930237-4) paper discussed in section. We will be implementing the LNP model described in this paper. Use the paper as a reference if needed.<br><br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

In [ ]:
# Load data
d = None
d = sio.loadmat('data_for_cell77.mat')
if d != None:
  print('Data loaded')

<br><br>
We will focus on the activity of a single neuron recorded while a mouse wanders around an arena. **The binsize is already set to 20 ms; we won't be varying this parameter.**

> `times` denotes the edges of each time bin
>
> `spikes` contains the number of spikes in each bin

<br>To track the mouse's position within the arena, we use the x/y positions of two LEDs on the head of the mouse (one on the left, one on the right). The mouse's location is calculated as the position of the center point between the two LEDs.

> `posx_c` is the x-position of the mouse's head
>
> `posy_c` is the y-position of the mouse's head


In [ ]:
bin_width = 0.02  # 20 ms
times = d['post']
times = np.append(times, times[-1] + bin_width)
spikes = d['spiketrain'][:, 0]

posx = d['posx'][:,0] # x-position of left LED every 20 ms
posx2 = d['posx2'][:,0] # x-position of right LED every 20 ms
posx_c = d['posx_c'][:, 0]  # x-position of mouse's head (middle of LEDs)

posy = d['posy'][:,0] # y-position of left LED every 20 ms
posy2 = d['posy2'][:,0] # y-position of right LED every 20 ms
posy_c = d['posy_c'][:, 0] # y-position of mouse's head (middle of LEDs)

box_size = d['boxSize'][0, 0]

orig_T = posx.shape[0]

<br><br>
From the LED data, we can also compute the `speed` and the `head_direction` of the mouse. We do this in the cell below.

In [ ]:
# From LED data...

# Compute speed
velx = np.diff(posx_c); vely = np.diff(posy_c); # add the extra just to make the vectors the same size
speed = np.sqrt(velx**2+vely**2) / bin_width;
speed = np.append(speed, speed[-1])

# Set any values over 50 cm/s to 50 cm/s
max_speed = 50
speed[speed > max_speed] = max_speed

# Compute head direction
head_direction = np.arctan2(posy2-posy, posx2-posx)+np.pi/2;
head_direction[head_direction < 0] += np.pi*2


<br><br>Let's plot our data. We have the spike occurences in 20 ms bins in `spikes`, the speed values in 20 ms bins in `speed`, the head direction values in 20 ms bins in `head_direction`, and the x/y positions of the mouse in `pos_x` and `pos_y` respectively (not shown).<br><br>

In [ ]:
fig, axes = plt.subplots(3, 1, figsize = (10, 5), sharex = True)

t = np.arange(0, orig_T*bin_width, bin_width)
axes[0].plot(t, spikes, 'k')
axes[0].set(title = 'Spikes per bin')

axes[1].plot(t, speed, 'k')
axes[1].set(title = 'Speed (cm/s)')

axes[2].plot(t, head_direction, 'k')
axes[2].set(title = 'Head direction (radians)', xlabel = 'Time (s)')

plt.tight_layout()

<br><br>
As you know, it's vital to use separate data for fitting the model and for testing its accuracy. Thus we will start by splitting our trials (time bins) into train and test subsets. We'll interleave the test chunks throughout the experiment.

We're actually going to use only 30% of our data total. This is to conserve memory and model fitting time on this homework (it's a lot of data!). In real life, you'd want to use all of your data for analyses, but it's often sensible to draft code using just a portion of data.

In [ ]:
# Use 30% of data total to conserve time/memory
T = int(.3*orig_T)

# Determine number of bins to use for test and train data
test_T = int(.2*T)      # 20% for testing
train_T = T - test_T    # 80% for training

# Get test bin indices
test_inds = []
for i_chunk in range(5):
  # in each fifth, take the first 20% for test data
  test_inds.append(np.arange(i_chunk*T/5, i_chunk*T/5+test_T/5) )

test_inds = np.concatenate(test_inds).astype('int')

# Define train bin indices as non-test ones
train_inds = np.setdiff1d(np.arange(T), test_inds)

# Splice spiking data into train vs test data
spikes_train = spikes[train_inds]
spikes_test = spikes[test_inds]

# Visualize train and test time bin assignment
plt.plot(train_inds,np.ones(train_inds.shape)-.1,'r.')
plt.plot(test_inds,np.ones(test_inds.shape),'b.')
plt.ylim([0, 2])
plt.xlabel('Time bin number')
plt.legend(['Train', 'Test']);

---

# I. Fitting an LNP model with head direction as input

Like Hardcastle et al., we want to fit a Linear-Nonlinear-Poisson model to predict spikes from the behavioral data. We will first build an LNP model with head direction as the only input.

Since we are using pretty big time bins (20 ms), we will use head direction during the same time bin as the stimulus to predict the neural response. In other words, we won't consider prior or future head directions, which rests on the assumption that the neuron's response latency is within 20 ms.

The LN model framework isn't set up to convert a single number (head direction) to a predicted response -- we wouldn't be able to interpret much from the fitted linear filter. We will do something quite common and bin the head direction values.

We'll create `hd_bins`, an array where the rows are time steps and the columns are different ranges of head direction values given by `hd_bin_edges`. The entries are 1 if the mouse's head direction was in that range of head directions at that time, 0 otherwise.

In [ ]:
# Bin head direction
n_hd_bins = 18
hd_bin_edges = np.linspace(0, 2*np.pi+.01, n_hd_bins + 1)

hd_bins = np.zeros((T, n_hd_bins))

for i_t in range(T):
    hd_bins[i_t, :], _ = np.histogram(head_direction[i_t], bins = hd_bin_edges)

# Split into train and test
hd_bins_train = hd_bins[train_inds]
hd_bins_test = hd_bins[test_inds]

## 1a (coding): Building an LNP model with head direction as input

Let's build our LNP model! 

We'll use an exponential nonlinearity. We can write our model as:

$$
\hat{r} = e^{X \cdot k}
$$
$$
\hat{y} = Poiss(\hat{r})
$$

where $X$ is a matrix in which the rows are samples/observations (different time bins) and the columns are different features of the input. In this case, each column would correspond to a different head direction bin.

$k$ is the linear filter we're learning and will be a vector the same size as the number of input features.

$\hat{r}$ is the predicted underlying response.

$\hat{y}$ is a predicted spike train.

<br><br>
Complete the code below to build a forward LNP model (predicting response and spikes given the head direction and linear filter).


In [ ]:
def LNP_predictions(linear_filter, inputs):
    """ Predict underlying response and spiking using LNP model

    Args:
    linear_filter: numpy array that is a vector of size (number of inputs, )
    inputs: numpy array that is size (number of time steps, number of inputs)

    Return:
      predicted_response: numpy array of vector of size (number of time steps,)
      predicted_spikes: numpy array of vector of size (number of time steps, )
    """
    
    # TO DO: Compute the filtered input
    # hint: consider the desired shape of filtered_values
    filtered_values = ...

    # TO DO: Apply exponential nonlinearity
    predicted_response = ...

    # TO DO: Draw Poisson spiking from predicted response
    # hint: https://numpy.org/doc/stable/reference/random/generated/numpy.random.poisson.html
    predicted_spikes = ...
    
    return predicted_response, predicted_spikes


# Define a placeholder head direction filter
hd_filter = np.zeros((n_hd_bins,))

# Predict response
predicted_response, predicted_spikes = LNP_predictions(hd_filter, hd_bins_train)


---

## 1b: Checking understanding

Let's pause and check our understanding. 

i) What is the shape of `predicted_response` and what does that correspond to? 

ii) First, without actually calculating it, what would be the accuracy of our model so far and why? 

iii) How would you determine accuracy in this case? Be specific. If you don't get an actual number for accuracy, why?

_You may want to explore the variables using code._

<font color=#2AAA8A><span style="font-size:larger;">
**Answer**


---

## 1c (coding): Calculating NLL for LNP model

We have set up our LNP model, and we now want to fit this model to the data by minimizing the negative log likelihood.

Complete the function below that will compute NLL for any given inputs `linear_filter`, `X`, and `spikes`.
<br>

In [ ]:
def compute_nll(linear_filter, X, spikes):

    # Predict response given filter and inputs
    predicted_rate, predicted_spikes = LNP_predictions(linear_filter, X)

    # TO DO: Compute negative log likelihood
    neg_log_lik = ...
    # hint: use the formula from class 6, but be thoughtful about what variables
    # you choose! what yields the most reliable estimate of error?)
    
    return neg_log_lik

negative_log_likelihood = compute_nll(hd_filter, hd_bins_train, spikes_train)
print(negative_log_likelihood)

---

## 1d (coding): Fitting the LNP model

To find the parameters that minimize a metric (in this case negative log likelihood), we can use `scipy.optimize.minimize`. To use this, we need an initial guess for the linear filter and our function that outputs the metric (`compute_nll`). The initial guess doesn't really matter for now (just guess all zeros or ones), it just needs to be the right size and shape.


In [ ]:
from scipy.optimize import minimize

# TO DO: Initial guess of linear filter
init_guess = ...

# Fit model
result = minimize(compute_nll, init_guess, args=(hd_bins_train, spikes_train), method="Powell")

print(result)
linear_filter = result['x']
NLL_H = result['fun']

<br><br>
**Let's visualize the fitted linear filter!**

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(hd_bin_edges[:-1], linear_filter, 'k')
ax.set(xlabel = 'Head direction (radians)');

# Your output should look like the plot in "HW3-P1d_HD-filter.png" found in the HW3 folder. 

---

## 1e: Intepreting the linear filter

i) What head direction of the mouse results in the highest neural response? What would we expect to see if the neuron wasn't influenced at all by the head direction of the mouse?

ii) Can you tell how much the neuron's response is affected by head direction from looking at this filter?

<font color=#2AAA8A><span style="font-size:larger;">
**Answer**

## 1f: Assessing model performance (coding)

To determine how much the neural responses are affected by head direction, we need to compute a metric of how good the model is -- how much of the variance in neural responses the model can explain. 

Remember, we don't want to assess the performance on the model based on the training data, but now want to use the data we set aside for testing the model.

Using the test data:

i) Predict the underlying response and spikes using your fitted LNP model.

ii) Modify the plotting code to show the predicted spikes on the second subplot, to compare with the true spikes.

iii) Compute the correlation coefficient between the predicted response and the true spikes. This is a common way to assess goodness of spiking predictions. A correlation coefficient of 0 means there is no correlation; the model is predicting spikes at chance levels. A correlation coefficient of 1 is perfect prediction.

In [ ]:
# i) TO DO: Predict underlying response and spikes
predicted_response, predicted_spikes = ...

# Visualize predictions vs real data
fig, axes = plt.subplots(2, 1)
axes[0].eventplot(np.where(spikes_test)[0], colors='k')
axes[0].set(title = 'True Spikes')

# ii) TO DO: add subplot to display predicted spikes
...
axes[1].set(title = 'Predicted Spikes')
plt.tight_layout()

In [ ]:
# iii) TODO: Compute the correlation coefficient between true and predicted responses on test data. 
#      Recall, above, when you considered which variables provide a stable estimate of accuracy. 
corr_coef = ...
print(f'The correlation coefficient is {corr_coef}')

## 1g: Interpret the result (conceptual)

Does it seem like the neuron is very driven by head direction? Are the neural responses explained well by what head direction the mouse is facing? Explain your thinking.

<font color=#2AAA8A><span style="font-size:larger;">
**Answer**


---

# II. Predicting response using multiple variables

**Fitting the LNP to head direction *and* running speed**

We now want to predict the neural response as a function of both speed and head direction during each time bin.

Like for head direction, we will group speed values into bins. In the cell below, the array `speed_bins` has rows corresponding to time bins and columns corresponding to different speed ranges.


In [ ]:
# Determine the number of bins
n_speed_bins = 10

# Get bin edges
max_speed = np.max(speed)
speed_bin_edges = np.linspace(0, max_speed, n_speed_bins + 1)

# Bin speed values
speed_bins = np.zeros((T, n_speed_bins))
for i_t in range(T):
    speed_bins[i_t, :], _ = np.histogram(speed[i_t], bins = speed_bin_edges)

# Splice data into test vs train
speed_bins_train = speed_bins[train_inds]
speed_bins_test = speed_bins[test_inds]

## 1h: Fitting an LNP model to both speed and head direction (coding)

Complete the code below to fit an LNP model to both speed and head direction.

Hints:
* You should change your input and initial guess but use the same functions you've already made.
* The input data is always structured as an array of [samples x features]. To use information from two variables in the input data, you will include additional features for each sample (time bin).

In [ ]:
# TO DO: complete code

# Make training and testing input data, including both speed and head direction
inputs_train = ...
inputs_test = ...

# Initial guess of linear filter
init_guess = ...

# Fit model
result = minimize(compute_nll, init_guess, args=(inputs_train, spikes_train), method="Powell")
print(result)
linear_filter = ...

# Get filters for speed and head direction
# hint: The linear filter contains coefficients for speed bins followed by head direction bins
speed_filter = ...
hd_filter = ...


# Visualize filters - separate out speed from head direction filters
fig, axes = plt.subplots(1, 2, figsize = (10, 5))

axes[0].plot(speed_bin_edges[:-1], speed_filter, 'k')
axes[0].set(xlabel = 'Speed (cm/s)');

axes[1].plot(hd_bin_edges[:-1], hd_filter, 'k')
axes[1].set(xlabel = 'Head direction (radians)');

## 1i: Assessing model performance (conceptual)

How good are the predictions when we used both head direction and speed as inputs to the model?


In [ ]:
# Predict response and spikes
predicted_response, predicted_spikes = LNP_predictions(linear_filter, inputs_test)

# Compute the correlation coefficient between true and predicted response on test data
corr_coef = np.corrcoef(predicted_response, spikes_test)[0, 1]
print(f'The correlation coefficient is {corr_coef}')


<br>**Would you classify this neuron as singly selective to head direction or as having mixed selectivity to head direction and speed? Explain your answer.**
<br> To rigorously determine the answer, we should perform a deeper statistical comparison. For the purpose of the assignment, make a cursory assessment using the results you've already computed.

<font color=#2AAA8A><span style="font-size:larger;">
**Answer**

<font color=#2AAA8A>

---

# III. Incorporating position as an input

It turns out that this particular neuron is actually affected a lot by position. In other words, the LNP's prediction of spiking responses is better if we include position as an input.


## 1j. Using position as an input to the LNP (coding)

**Write code to determine how this neuron's spiking is affected by position. What is the prediction accuracy with position alone, and with position + head direction?**

Note there is an additional consideration in setting up the filter for the position of the mouse. The position inputs should reflect the combined x,y position of the mouse instead of the x pos and y pos separately. If you look at a grid cell's RF, it's clear that neurons are selective for a location in space, i.e. a certain *combination* of x and y. Therefore, we want to bin position data within the whole grid. If you choose 10 bins for each of the x and y dimensions, then you'd have 100 bins total (instead of 20).

Overall, this question will be more of a challenge. Take it step by step, and recycle code from above.

In [ ]:
# First, compute position histogram, linking x and y positions
nposbins = 10  # in each x and y dimensions
pos = np.column_stack((posx_c, posy_c))
posbins = np.arange(box_size/nposbins/2, box_size, box_size/nposbins)
posgrid = np.zeros((T, nposbins**2))

for i_t in range(T):
    xcoor = np.argmin(np.abs(pos[i_t,0]-posbins))
    ycoor = np.argmin(np.abs(pos[i_t,1]-posbins))
    bin_idx = np.ravel_multi_index(([nposbins - ycoor - 1], [xcoor]), (nposbins, nposbins))
    posgrid[i_t, bin_idx] = 1
    
print("posgrid shape is "+ str(np.shape(posgrid)))


In [ ]:
#####Your code here! Feel free to break this up into separate cells.

# Splice data into test vs train
pos_train = ...
pos_test = ...

print("pos_train shape is "+ str(np.shape(pos_train)))


# Fit model 


# Plot position filter in 1d and in 2d. Neither will be immediately interpretable, but read on.
position_filter = ...



In [ ]:
# The resulting position filter should look like this:
position_filter_key = np.load('pos_filter.npy')

In [ ]:
# Why does the filter look so weird? 
# The mouse does not travel to every position in the grid within this
# subset of the experimental data. This results in the highly negative
# values you see in the filter above. These artifacts don't impact the
# prediction accuracy much in this case, but it makes it impossible to visualize
# any structure in the filter. There are a few ways we can correct for this,
# including making fewer/larger position bins, or setting the filter values for
# the missing positions to the median filter value.
position_filter_key[position_filter_key < -70] = np.median(position_filter_key[position_filter_key > -70])


# Plot linear filter for position in 1d (still doesn't make much sense)
binvecidx = np.arange(0,len(position_filter_key))
fig, ax = plt.subplots(1, 1)
ax.plot(binvecidx, position_filter_key, 'k')
ax.set(xlabel = 'pos grid bins');

# Plot linear filter in 2d (see a little more structure)
linearfilter_posgrid = np.reshape(position_filter_key, (nposbins, nposbins))
fig, ax = plt.subplots()
cax = ax.imshow(linearfilter_posgrid, cmap='gray')
fig.colorbar(cax)
plt.show()


**Use `position_filter_key` as your linear filter for the next cell if you couldn't get your filter to match it.**

In [ ]:
# Use position_filter_key as your linear filter for this cell.

# Test model
...
corr_coef_pos = ...
print(f'The correlation coefficient using position information only is {corr_coef_pos}.')


# ~~~~~~ Comments on position-only result ~~~~~~
#
# Model performance is substantially better (.178) but still not great.
# It gets much better when more of the data is used.
# By using only 30% of the data, many of the position bins are visited only
# a few times and this contributes to relatively poor model accuracy.
#
# We can fudge having more data by increasing the size of the position bins.
# The caveat is that the bins have to be small enough to capture the positional
# tuning width of the cell (ie. the grid cell's hotspots can't be closer
# together than the bins can capture).


In [ ]:
################################################################
####     Now, fit a model to position AND head direction    ####
################################################################
# (forget about speed because it doesn't contribute to this cell)
# Note: it may take a little longer to fit the model this time (up to 10s).

...


corr_coef_pos_hd = ...
print(f'The correlation coefficient using position and head direction is {corr_coef_pos_hd}.')


In [ ]:
# ~~~~~~ Interpreting the result ~~~~~~
#
# Including both head direction and position increases model performance to 0.197, 
# the highest correlation we've yet observed. This may not seem impressive, but it 
# corresponds to the range of performances observed by Hardcastle. 
#   -> See the middle histogram in Fig. S3a (code below will open an image). 
# Note that in the paper they also added a smoothing step before calculating the 
# correlation coefficient, which would slightly improve performance.

from PIL import Image
img = Image.open('Hardcastle_FigS3a.png')
img.show() 

---

# IV. Alternative methods

## 1k: Using the LNP vs. tuning curves (conceptual)

Why are tuning curves insufficient to determine if neurons have mixed selectivity? What, specifically, can you learn from the present approach that you could not by plotting tuning curves for each stimulus property?


<font color=#2AAA8A><span style="font-size:larger;">
**Answer**

---